In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import numpy as np
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os

# Disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"

# Load the dataset
file_path = "5100GPTDatasetHITL.csv.csv"
df = pd.read_csv(file_path)

# Prepare data
data = pd.DataFrame({
    "text": df["H/T"].astype(str).tolist() + df["CS"].astype(str).tolist(),
    "label": [0] * len(df) + [1] * len(df)  # 0 = HATE, 1 = COUNTER
}).sample(frac=1.0, random_state=42).reset_index(drop=True)

# Train-test split
train_df, test_df = train_test_split(data, test_size=0.2, stratify=data["label"])

# Tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

# Convert to Hugging Face Datasets
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)
train_ds = train_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)

# Model
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

Map:   0%|          | 0/8160 [00:00<?, ? examples/s]

Map:   0%|          | 0/2040 [00:00<?, ? examples/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-5-ffd65a5cc17f>:68: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train and save
trainer.train()
model.save_pretrained("saved_model")
tokenizer.save_pretrained("saved_model")

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.042200,0.029074,0.994608,0.994579,1.000000,0.989216
2,0.031700,0.015781,0.996078,0.996063,1.000000,0.992157
3,0.021900,0.029515,0.996078,0.996063,1.000000,0.992157
4,0.019400,0.032024,0.996078,0.996090,0.993177,0.999020
5,0.002900,0.032520,0.995098,0.995084,0.998028,0.992157
6,0.006900,0.023788,0.996569,0.996567,0.997056,0.996078
7,0.013600,0.027759,0.996569,0.996570,0.996082,0.997059
8,0.002000,0.030407,0.997059,0.997056,0.998035,0.996078
9,0.000000,0.025068,0.997549,0.997548,0.998037,0.997059
10,0.002600,0.009559,0.999020,0.999020,0.999020,0.999020


TrainOutput(global_step=20400, training_loss=0.009699382022939901, metrics={'train_runtime': 1322.5866, 'train_samples_per_second': 123.395, 'train_steps_per_second': 15.424, 'total_flos': 4771383322315680.0, 'train_loss': 0.009699382022939901, 'epoch': 20.0})

('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/sentencepiece.bpe.model',
 'saved_model/added_tokens.json')

In [ ]:
import shutil

# Zip the folder
shutil.make_archive('saved_model', 'zip', 'saved_model')

# Download the zipped folder
from google.colab import files
files.download('saved_model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 77.1 MB/s eta 0:00:00


In [ ]:
# Convert your dataset to JSONL format suitable for fine-tuning

output_path = "fine_tune_data.jsonl"

# Filter and rename
fine_tune_data = df[["H/T", "CS"]].dropna()
fine_tune_data = fine_tune_data.rename(columns={"H/T": "instruction", "CS": "response"})

# Format instructions
fine_tune_data["instruction"] = fine_tune_data["instruction"].apply(
    lambda x: f"ഹേറ്റുകൾ: {x.strip()}"
)

# Export as JSONL
fine_tune_data.to_json(output_path, orient="records", lines=True, force_ascii=False)

# Display path for download
output_path


'fine_tune_data.jsonl'

In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes

In [ ]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate peft


In [ ]:
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd
import os
import torch

# Disable WandB
os.environ["WANDB_DISABLED"] = "true"

# ✅ Load the dataset
df = pd.read_json("fine_tune_data.jsonl", lines=True)
dataset = Dataset.from_pandas(df)

# Optional: Remove index column
if "__index_level_0__" in dataset.column_names:
    dataset = dataset.remove_columns("__index_level_0__")

# ✅ Format as instruction prompt
def format_prompt(example):
    return f"വാക്യത്തിൽ തെളിയിക്കുന്ന വെറുപ്പിന് എതിരായ ഒരു മറുപടി എഴുതുക.\n\n{example['instruction']}\n\nമറുപടി: {example['response']}"

dataset = dataset.map(lambda x: {"text": format_prompt(x)})

# ✅ Load tokenizer and model
model_name = "VinkuraAI/KunoRZN-Llama-3-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map="auto")

# ✅ Tokenization with labels
def tokenize(example):
    result = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(tokenize, batched=True)

# ✅ Apply QLoRA configuration
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="kuno-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    save_strategy="epoch",
    logging_dir="./logs",
    save_total_limit=2,
)

# ✅ Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

# ✅ Train!
trainer.train()

# ✅ Save fine-tuned model
model.save_pretrained("kuno-finetuned")
tokenizer.save_pretrained("kuno-finetuned")


Map:   0%|          | 0/5100 [00:00<?, ? examples/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/5100 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-7-fd561ea8c940>:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_r

Step,Training Loss
500,0.225400
1000,0.084900


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
500,0.225400
1000,0.084900
1500,0.051100


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


('kuno-finetuned/tokenizer_config.json',
 'kuno-finetuned/special_tokens_map.json',
 'kuno-finetuned/chat_template.jinja',
 'kuno-finetuned/additional_chat_templates/tool_use.jinja',
 'kuno-finetuned/tokenizer.json')

In [ ]:

# ---- SSF-Based Counter-Speech Chatbot with KunoRZN-Llama-3-3B Generation ----

# Required installations (uncomment in Colab)
# !pip install transformers sentence-transformers faiss-cpu

import torch
import faiss
import numpy as np
import pandas as pd
import math
from sklearn.metrics.pairwise import cosine_similarity
from transformers import (
    XLMRobertaTokenizer, XLMRobertaForSequenceClassification,
    AutoTokenizer, AutoModelForCausalLM
)
from sentence_transformers import SentenceTransformer



# Load dataset
df = pd.read_csv("5100GPTDatasetHITL.csv.csv")
cs_list = df["CS"].astype(str).tolist()

# === 1. Semantic Similarity ===
embedder = SentenceTransformer("sentence-transformers/LaBSE")
cs_embeddings = embedder.encode(cs_list, convert_to_numpy=True, show_progress_bar=True)
index = faiss.IndexFlatL2(cs_embeddings.shape[1])
index.add(cs_embeddings)

def retrieve_topk_semantic(hs_text, top_k=30):
    hs_embedding = embedder.encode([hs_text], convert_to_numpy=True)
    distances, indices = index.search(hs_embedding, top_k)
    return [(cs_list[i], i) for i in indices[0]]

# === 2. Stance-Based Ranking ===
# Load stance classification model
stance_model_path = "./saved_model"
stance_tokenizer = XLMRobertaTokenizer.from_pretrained(stance_model_path)
stance_model = XLMRobertaForSequenceClassification.from_pretrained(stance_model_path)

# stance_tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
# stance_model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base")
stance_model.eval()
stance_model.to("cuda" if torch.cuda.is_available() else "cpu")

def get_cls_embedding(text):
    inputs = stance_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(stance_model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = stance_model.base_model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

def rank_by_stance(hs_text, retrieved_cs, top_k=10):
    hs_vec = get_cls_embedding(hs_text)
    cs_vecs = [get_cls_embedding(cs) for cs, _ in retrieved_cs]
    sims = cosine_similarity([hs_vec], cs_vecs)[0]
    sorted_indices = np.argsort(sims)
    return [retrieved_cs[i][0] for i in sorted_indices[:top_k]]

# === 3. Fitness (Perplexity) ===
from transformers import AutoModelForSeq2SeqLM
gen_tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
gen_model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")
gen_model.eval()
gen_model.to("cuda" if torch.cuda.is_available() else "cpu")

def compute_perplexity(prompt):
    encodings = gen_tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=128)
    input_ids = encodings.input_ids.to(gen_model.device)
    with torch.no_grad():
        outputs = gen_model(input_ids=input_ids, labels=input_ids)
        loss = outputs.loss
    return math.exp(loss.item()) if loss is not None else float("inf")

def rank_by_fitness(hs_text, cs_candidates, top_k=10):
    prompt_template = lambda cs: f"{hs_text} അതിനെ എതിർക്കുന്നു: {cs}"
    ranked = [(cs, compute_perplexity(prompt_template(cs))) for cs in cs_candidates]
    return sorted(ranked, key=lambda x: x[1])[:top_k]

# === 4. Final Generation using VinkuraAI/KunoRZN-Llama-3-3B ===
# kuno_tokenizer = AutoTokenizer.from_pretrained("VinkuraAI/KunoRZN-Llama-3-3B")
# kuno_model = AutoModelForCausalLM.from_pretrained("VinkuraAI/KunoRZN-Llama-3-3B")
kuno_model = AutoModelForCausalLM.from_pretrained("kuno-finetuned")
kuno_tokenizer = AutoTokenizer.from_pretrained("kuno-finetuned")

kuno_model.eval()
kuno_model.to("cuda" if torch.cuda.is_available() else "cpu")

def generate_final_response(hs_text, top_cs):
    prompt = f"""ദയവായി ഒരു ശാന്തമായ, ബഹുമാനപരമായ, വിശദീകരണപരമായ മറുപടി തയ്യാറാക്കുക.
താഴെയുള്ള വെറുപ്പ് പ്രകടിപ്പിക്കുന്ന വാക്യത്തിന്ഒ, ഒരു അറിവ് അടിസ്ഥാനമാക്കിയുള്ള എതിര്‍പ് രേഖപ്പെടുത്തുക.
മറുപടി തർക്കം ക്ഷണിക്കാതെ, യുക്തിപൂർണ്ണവും മാന്യവുമായിരിക്കുക.

ഹേറ്റുകൾ: {hs_text}
ഉദാഹരണ പ്രതികരണം: {top_cs}

മറുപടി:"""
    inputs = kuno_tokenizer(prompt, return_tensors="pt").to(kuno_model.device)
    with torch.no_grad():
        output_ids = kuno_model.generate(
            **inputs,
            max_new_tokens=255,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7
        )
    return kuno_tokenizer.decode(output_ids[0], skip_special_tokens=True).split("മറുപടി:")[-1].strip()

# === Full Chatbot Function ===
def generate_counter_speech(hs_input):
    sem_candidates = retrieve_topk_semantic(hs_input, top_k=30)
    stance_filtered = rank_by_stance(hs_input, sem_candidates, top_k=10)
    top_fluent = rank_by_fitness(hs_input, stance_filtered, top_k=1)
    print(f"Top Fluent: {top_fluent}")
    if not top_fluent:
        return "ക്ഷമിക്കണം, അതിനായുള്ള മറുപടി കണ്ടെത്താൻ കഴിഞ്ഞില്ല."
    return generate_final_response(hs_input, top_fluent[0][0])

# # === Example Use ===
# user_input = "അവർക്ക് വിവാഹം കഴിക്കാനുള്ള അവകാശമില്ല"
# response = generate_counter_speech(user_input)
# print(f"🤖: {response}")


Batches:   0%|          | 0/160 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
user_input = "അവർക്ക് വിവാഹം കഴിക്കാനുള്ള അവകാശമില്ല"
response = generate_counter_speech(user_input)
print(f"🤖: {response}")

Top Fluent: [('അതുകൊണ്ടുതന്നെ, അവർ നിങ്ങൾക്ക് ഒരു പ്രശ്നവും ഉണ്ടാക്കാത്ത കാലത്തോളം, മറ്റുള്ളവർ എങ്ങനെ ജീവിക്കണം എന്ന് നിർദേശിക്കാൻ നിങ്ങൾക്ക് യാതൊരു അവകാശവുമില്ല. ഓരോ വ്യക്തിക്കും അവരുടെ ജീവിതം തങ്ങൾക്ക് ഇഷ്ടമുള്ള രീതിയിൽ നയിക്കാൻ സ്വാതന്ത്ര്യമുണ്ട്. "ജീവിക്കാൻ അനുവദിക്കില്ല" എന്ന് കമൻഡ് അടിക്കാൻ ആരും അവകാശവാനല്ല. ഒരു ജനാധിപത്യ സമൂഹത്തിൽ, എല്ലാ വ്യക്തികളെയും ബഹുമാനിക്കുകയും, അവരുടെ അവകാശങ്ങളെ അംഗീകരിക്കുകയും ചെയ്യേണ്ടതുണ്ട്. എല്ലാവരും സമത്വത്തോടെ ജീവിക്കേണ്ടതാണെന്നത് മനസ്സിലാക്കുക എന്നതാകും ഏറ്റവും ഉചിതം.', 2513847249930.3887)]
🤖: എന്തായാലും അവർ ജീവിക്കാം അല്ലേ, അവർക്ക് ജീവിതം ഉണ്ട്. അവർക്ക് മനുഷ്യരാണെന്ന് സാധാരണക്കാർ തയ്യേറ്റാവുന്നു, കാരണം അവർ സാമൂഹികമായി അവരോട് അവരെ അവരുടെ പ്രകൃ�


In [ ]:
print(f"🤖: {response[1]}")

🤖: അവർ ഇങ്ങനെ പ്രവൃത്തിക്കുന്നത് അവരുടെ സാമൂഹിക സ്ഥിതിയെ തകർുന്നു, അതുമാത്രമുള്ള സംസ്കാരം നമുക്ക് സാദൃക്കരിക്കാനും ഇല്ല. അവർക്കും അവകാശം ലളിതമായും നീങ്ങിപ്പ�


In [ ]:
!pip install detoxify

In [ ]:
from detoxify import Detoxify
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
nltk.download('punkt_tab')


# Load Detoxify
tox_model = Detoxify("original")
# generated_cs = "നിനിക്ക് അതുകൊണ്ടുതന്നെ, അവർ നിങ്ങൾക്ക് അത്രയിലേറ്റിൽ ആഗ്രഹിക്കുന്ന വിധത്തിൽ ജീവിക്കാനുള്ള അവകാശവും ഉള്ളതായിരിക്കുകയില്ല. ഒരു വിവാഹത്തിൽ പങ്കെടുക്കുന്നത് അവ"

def clean_generated_text(text):
    if "." in text:
        return text.rsplit(".", 1)[0] + "."
    return text  # fallback

safe_cs = clean_generated_text(response)

# ---- Evaluation Functions ----
def compute_relevance(hs_text, cs_text):
    hs_vec = embedder.encode([hs_text])
    cs_vec = embedder.encode([cs_text])
    return cosine_similarity(hs_vec, cs_vec)[0][0]

def compute_novelty(cs_text, training_cs_list):
    cs_vec = embedder.encode([cs_text])
    train_vecs = embedder.encode(training_cs_list)
    sims = cosine_similarity(cs_vec, train_vecs)[0]
    return 1 - np.max(sims)

def compute_bleu(reference, candidate):
    ref_tokens = [nltk.word_tokenize(reference)]
    cand_tokens = nltk.word_tokenize(candidate)
    return sentence_bleu(ref_tokens, cand_tokens, smoothing_function=SmoothingFunction().method1)

def compute_toxicity(text):
    return tox_model.predict(text).get("toxicity", 0.0)

def compute_fluency(text):
    inputs = gen_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(gen_model.device)
    with torch.no_grad():
        outputs = gen_model(input_ids=inputs["input_ids"], labels=inputs["input_ids"])
        loss = outputs.loss
    return np.exp(loss.item()) if loss is not None else float("inf")
# def compute_fluency(text):
#     if len(text.strip()) < 10:
#         return float("inf")  # Too short to evaluate

#     prompt = f"അതിനെ എതിർക്കുന്നു: {text.strip()}"
#     inputs = gen_tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=128).to(gen_model.device)

#     with torch.no_grad():
#         outputs = gen_model(input_ids=inputs["input_ids"], labels=inputs["input_ids"])
#         loss = outputs.loss

#     if loss is None or loss.item() > 20:  # cap on extreme values
#         return float("inf")

#     return np.exp(loss.item())


def predict_stance(text):
    inputs = stance_tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(stance_model.device)
    with torch.no_grad():
        logits = stance_model(**inputs).logits
    probs = torch.softmax(logits, dim=-1)
    label = torch.argmax(probs, dim=-1).item()
    return "COUNTER" if label == 1 else "HATE", probs[0][1].item()

# ---- Evaluate the generated response ----
generated_cs = response
# reference_cs = "മാനവ അവകാശങ്ങൾ എല്ലാവർക്കും ബാധകമാണ്."  # Optional if you have a gold response
referece_cs = 'അതുകൊണ്ടുതന്നെ, അവർ നിങ്ങൾക്ക് ഒരു പ്രശ്നവും ഉണ്ടാക്കാത്ത കാലത്തോളം, മറ്റുള്ളവർ എങ്ങനെ ജീവിക്കണം എന്ന് നിർദേശിക്കാൻ നിങ്ങൾക്ക് യാതൊരു അവകാശവുമില്ല. ഓരോ വ്യക്തിക്കും അവരുടെ ജീവിതം തങ്ങൾക്ക് ഇഷ്ടമുള്ള രീതിയിൽ നയിക്കാൻ സ്വാതന്ത്ര്യമുണ്ട്. "ജീവിക്കാൻ അനുവദിക്കില്ല" എന്ന് കമൻഡ് അടിക്കാൻ ആരും അവകാശവാനല്ല. ഒരു ജനാധിപത്യ സമൂഹത്തിൽ, എല്ലാ വ്യക്തികളെയും ബഹുമാനിക്കുകയും, അവരുടെ അവകാശങ്ങളെ അംഗീകരിക്കുകയും ചെയ്യേണ്ടതുണ്ട്. എല്ലാവരും സമത്വത്തോടെ ജീവിക്കേണ്ടതാണെന്നത് മനസ്സിലാക്കുക എന്നതാകും ഏറ്റവും ഉചിതം.'

metrics = {
    "Relevance": compute_relevance(user_input, generated_cs),
    "Novelty": compute_novelty(generated_cs, [cs for cs, _ in retrieve_topk_semantic(user_input, top_k=30)]),
    "Fluency_PPL": compute_fluency(safe_cs),
    "Toxicity": compute_toxicity(generated_cs),
    # "BLEU": compute_bleu(reference_cs, generated_cs),
}

stance_label, stance_conf = predict_stance(generated_cs)
metrics["Stance"] = stance_label
metrics["Stance_Confidence"] = stance_conf

# Display metrics
for k, v in metrics.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


TypeError: expected string or bytes-like object, got 'list'

In [ ]:
print("Generated Counter-Speech:")
print(generated_cs)


Generated Counter-Speech:
നിനിക്ക് അതുകൊണ്ടുതന്നെ, അവർ നിങ്ങൾക്ക് അത്രയിലേറ്റിൽ ആഗ്രഹിക്കുന്ന വിധത്തിൽ ജീവിക്കാനുള്ള അവകാശവും ഉള്ളതായിരിക്കുകയില്ല. ഒരു വിവാഹത്തിൽ പങ്കെടുക്കുന്നത് അവ


In [ ]:
def compute_fluency(text):
    if len(text.strip()) < 10:
        return float("inf")  # Too short to evaluate

    prompt = f"അതിനെ എതിർക്കുന്നു: {text.strip()}"
    inputs = gen_tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=128).to(gen_model.device)

    with torch.no_grad():
        outputs = gen_model(input_ids=inputs["input_ids"], labels=inputs["input_ids"])
        loss = outputs.loss

    if loss is None or loss.item() > 20:  # cap on extreme values
        return float("inf")

    return np.exp(loss.item())


In [ ]:
!pip list --not-required

Package                               Version
------------------------------------- ------------
albumentations                        2.0.7
altair                                5.5.0
astropy                               7.1.0
autograd                              1.8.0
betterproto                           2.0.0b6
bigframes                             2.4.0
bigquery-magics                       0.9.0
bitsandbytes                          0.46.0
Bottleneck                            1.4.2
build                                 1.2.2.post1
cmake                                 3.31.6
community                             1.0.0b1
cudf-polars-cu12                      25.2.2
cufflinks                             0.17.3
cuml-cu12                             25.2.1
cvxopt                                1.3.2
cvxpy                                 1.6.5
cyipopt                               1.5.0
Cython                                3.0.12
dataproc-spark-connect                0.7.4
datasci